In [1]:
import os
import numpy as np
import pandas as pd
import math
import warnings
import time as time
points = pd.read_csv(os.path.join("data","points.csv"), sep=",")

In [7]:
#Clase para todas las funciones necesarias para el algoritmo de Weiszfeld
class weiszfeld:
    
    #Constructor
    def __init__(self,imagen= 0, Tweisz = 0, gradiente = 0, mini = 0):
        self.imagen = imagen
        self.Tweisz = Tweisz
        self.gradiente = gradiente
        self.mini = mini
    
    #Evaluacion de la función de Weber W(x,y)
    def fun(self ,x, points):
        self.imagen = 0
        for i in range(0,len(points)):
            x_i = np.array([points["x"][i], points["y"][i]])
            self.imagen += points["peso"][i]*np.linalg.norm(x-x_i)
        return self.imagen
    
    #Evaluacion del Operador de Weiszfeld
    def T_weisz(self, x , points):
        self.Tweisz = 0
        numerador = np.array([0,0])
        denominador = 0
        for i in range(0,len(points)):
            a_i = np.array([points["x"][i],points["y"][i]])
            if np.array_equal(x,a_i):
                return a_i
            w_i = points["peso"][i]
            numerador = numerador + (w_i/ np.linalg.norm(x-a_i)) * a_i
            denominador += w_i/ np.linalg.norm(x-a_i)
        self.Tweisz = numerador / denominador
        return self.Tweisz
    
    #Gradiente de la función de Weber
    def weisz_grad(self, x,points):
        self.gradiente = 0
        suma = np.array([0,0])
        for i in range(0,len(points)):
            a_i = np.array([points["x"][i],points["y"][i]]) 
            w_i = points["peso"][i]
            suma = suma + (w_i/ np.linalg.norm(x-a_i)) * (x-a_i)
        self.gradiente = suma
        return self.gradiente
    
    #Minimo producto punto entre el gradinte de weisfeld y los vertices fijos
    def minimo(self, x, points):
        self.mini = 0
        suma = np.array([0,0])
        producto = np.array([])
        for i in range(0,len(points)):
            a_i = np.array([points["x"][i],points["y"][i]]) 
            w_i = points["peso"][i]
            suma = suma + (w_i/ np.linalg.norm(x-a_i)) * (x-a_i)
        
        for i in range(0,len(points)):
            a_i = np.array([points["x"][i],points["y"][i]]) 
            dot_i = np.dot(suma, a_i)
            producto = np.append(producto,dot_i)
            
        self.mini = producto.min()
        return self.mini
    

def Weiszfeld(w,             #Clase de funciones para el algoritmo
              points,        #Dataframe con los datos del problema
              x_init = None, #Punto inicial
              opt = None     #Diccionario con tolerancia y nro máximo de iteraciones
             ):
    #default inputs
    if opt is None: opt= {'tol': 1e-4, 'iter': 500}
    if x_init is None: #Punto inicial optimo de Weiszfeld
        
        mini = 10.0**20 #numero arbitrariamente grande
        a_min= np.array([0,0])
        j = 0
        for i in range(0,len(points)):
            a_i = np.array([points["x"][i],points["y"][i]])
            w_i = w.fun(a_i,points)
            if w_i < mini : 
                mini = w_i              #minimo W(a_i)
                a_min = a_i             #vertice con el cual W(a_i) es minimo
                j=i                     #indice asociado a a_i tal que W(a_i) es minimo
                w_k = points["peso"][i] #Peso asociado a a_i tal que W(a_i) es minimo
        
        x = a_min 
        c = 1.5 #Constante entre 1 y 2
        sk = 0
        gradk = np.array([0,0])
        for i in range(0,len(points)):
            if i == j:
                pass
            else:
                w_i = points["peso"][i]
                a_i = np.array([points["x"][i],points["y"][i]])
                sk += w_i / np.linalg.norm(a_min-a_i)
                gradk = gradk + (w_i/np.linalg.norm(x-a_i)) * (a_min - a_i)
        G_k = gradk - (w_k/np.linalg.norm(gradk)) * gradk 

        x = a_min - (c / sk) * G_k #Definicion de punto inicial óptimo para la sucesion de Weiszfeld
    else : 
        x = x_init #Punto incial pasado por parametro
    
    # Parametros algoritmicos
    tol      = opt['tol']  #Tolerancia
    max_iter = opt['iter'] #Nro maximo de iteraciones
    
    print('Running Weiszfeld...');
    
    timing = np.zeros(max_iter) #array de tiempo por iteracion
    criter = np.zeros(max_iter) #Array de evaluación funcion objetivo por iteracion
    
    # Loop algoritmico
    for it in range(max_iter):
    
        t = time.time()
    
        # Actualizacion de x_k:= x en cada iteración 
        x_old = x;
        x = w.T_weisz(x,points)

        # Actualizacion de tiempo de ejecucion y de funcion objetivo
        timing[it] = time.time() - t
        criter[it] = w.fun(x ,points)
           
        # Criterio de parada
        D_r = w.fun(x_old, points)-np.dot(w.weisz_grad(x_old, points),x_old) + w.minimo(x_old,points)
        if np.abs((w.fun(x_old, points)-D_r)/ D_r) <= tol  and it>10:
            break
        
        print( str(it) + ' de un maximo de ' + str(max_iter) + ' iteraciones' )

    criter = criter[0:it+1]
    timing = np.cumsum(timing[0:it+1])
    
    return x, it, timing, criter
w = weiszfeld()
Weiszfeld(w,points)

Running Weiszfeld...
0 de un maximo de 500 iteraciones
1 de un maximo de 500 iteraciones
2 de un maximo de 500 iteraciones
3 de un maximo de 500 iteraciones
4 de un maximo de 500 iteraciones
5 de un maximo de 500 iteraciones
6 de un maximo de 500 iteraciones
7 de un maximo de 500 iteraciones
8 de un maximo de 500 iteraciones
9 de un maximo de 500 iteraciones
10 de un maximo de 500 iteraciones
11 de un maximo de 500 iteraciones
12 de un maximo de 500 iteraciones
13 de un maximo de 500 iteraciones
14 de un maximo de 500 iteraciones
15 de un maximo de 500 iteraciones


(array([394.15476007, 224.64950489]),
 16,
 array([0.00199366, 0.00299025, 0.00398874, 0.00498605, 0.00598431,
        0.00694847, 0.0089438 , 0.01093841, 0.01290107, 0.01389837,
        0.01489568, 0.01589322, 0.01689076, 0.01788712, 0.01885033,
        0.01981091, 0.02080774]),
 array([1690.73035476, 1690.47520956, 1690.34075918, 1690.27153801,
        1690.23644138, 1690.21882206, 1690.21003395, 1690.2056695 ,
        1690.20350827, 1690.20244017, 1690.20191302, 1690.20165309,
        1690.20152501, 1690.20146192, 1690.20143086, 1690.20141557,
        1690.20140804]))